In [1]:
import os
import torch
from torch import optim, nn, utils, Tensor
import pandas as pd
import torch.nn.functional as F
import pytorch_lightning as pl
torch.manual_seed(0)

/home/exitialium/miniconda3/envs/simpleNNstock/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
device = "cuda" if torch.cuda.is_available() else "cpu"

class SimpleNN(pl.LightningModule):
    def __init__(self):
        super().__init__()
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(104 * 22, 500)
        self.fc2 = nn.Linear(500, 100)
        self.fc3 = nn.Linear(100, 20)
        self.fc4 = nn.Linear(20, 5)
        self.fc5 = nn.Linear(5, 2)
        self.soft = nn.LogSoftmax(dim = 0)
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        x = torch.flatten(x)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        x = self.soft(x)
        loss = nn.NLLLoss(x,y)
        return loss

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr = 1e-3)
        return optimizer

    def forward(self, x):
        x = torch.flatten(x)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        x = self.soft(x)
        return x

In [9]:
simpleNN = SimpleNN().to(device)
print(simpleNN)

SimpleNN(
  (fc1): Linear(in_features=2288, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=100, bias=True)
  (fc3): Linear(in_features=100, out_features=20, bias=True)
  (fc4): Linear(in_features=20, out_features=5, bias=True)
  (fc5): Linear(in_features=5, out_features=2, bias=True)
  (soft): LogSoftmax(dim=0)
)


In [12]:
directory = 'processedDataset/'
X = list()
Y = list()
for filename in os.listdir(directory)[0:10]:
    f = os.path.join(directory, filename)
    X.append(torch.load(f))
    Y.append(filename.split('-')[2].replace(".pt",""))
dataset = list(zip(X,Y))

In [13]:
print(dataset)

[(tensor([[4.8000e+00, 4.6321e+07, 1.1880e+01,  ..., 2.1266e+03, 2.0066e+03,
         1.0028e+00],
        [4.8200e+00, 3.4499e+07, 1.1380e+01,  ..., 2.1266e+03, 2.0101e+03,
         1.0017e+00],
        [4.4500e+00, 2.8438e+07, 1.1380e+01,  ..., 2.1266e+03, 2.0129e+03,
         1.0014e+00],
        ...,
        [6.2770e+00, 3.7365e+07, 1.0230e+01,  ..., 2.8729e+03, 2.6907e+03,
         9.9995e-01],
        [6.2480e+00, 3.2417e+07, 1.0230e+01,  ..., 2.8729e+03, 2.6878e+03,
         9.9895e-01],
        [6.3350e+00, 3.3851e+07, 1.0230e+01,  ..., 2.8729e+03, 2.6880e+03,
         1.0001e+00]], dtype=torch.float64), '0'), (tensor([[6.6500e+00, 8.1790e+04, 1.1500e+01,  ..., 2.9456e+03, 2.7923e+03,
         1.0028e+00],
        [6.3800e+00, 8.0613e+04, 1.1500e+01,  ..., 2.9456e+03, 2.7978e+03,
         1.0020e+00],
        [6.1100e+00, 8.2626e+04, 1.1500e+01,  ..., 2.9456e+03, 2.7982e+03,
         1.0001e+00],
        ...,
        [3.2700e+00, 9.8319e+04, 1.1500e+01,  ..., 4.1855e+03, 3.8800

In [15]:
train, test = torch.utils.data.random_split(dataset, [9,1])

In [21]:
train_loader = torch.utils.data.DataLoader(train, 64, shuffle = True)
test_loader = torch.utils.data.DataLoader(test, 64)